In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
from PIL import Image
from torch.utils.data import Dataset
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm

In [5]:
datadir = '/content/drive/MyDrive/Facultate Informatica/Profesor/2024 - 2025/Biometrie/Curs/ColabMount/DATA/test_arc_images'
files = os.listdir(datadir)
print(files)

['class1_Arc_0030.png', 'class1_Arc_0032.png', 'class1_Arc_0022.png', 'class1_Arc_0010.png', 'class1_Arc_0001.png', 'class1_Arc_0010_v1.png', 'class1_Arc_0010_v2.png', 'class1_Arc_0010_v3.png']


In [17]:
model_file = '/content/drive/MyDrive/Facultate Informatica/Profesor/2024 - 2025/Biometrie/Curs/ColabMount/Models/fingerprint_classifier.pth'
mfiles = os.listdir(model_file)
print(mfiles)

NotADirectoryError: [Errno 20] Not a directory: '/content/drive/MyDrive/Facultate Informatica/Profesor/2024 - 2025/Biometrie/Curs/ColabMount/Models/fingerprint_classifier.pth'

In [6]:
class FingerprintClassificationDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.label_map = {
            'class1_Arc': 0,
            'class2_Whorl': 1,
            'class3_Loop': 2
        }

        for class_name, label in self.label_map.items():
            class_dir = os.path.join(root_dir, class_name)
            if os.path.isdir(class_dir):
                for file in os.listdir(class_dir):
                    if file.endswith(('.png', '.jpg', '.jpeg')):
                        self.image_paths.append(os.path.join(class_dir, file))
                        self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('L')
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label


In [7]:
class ClassificationLoss(nn.Module):
    def __init__(self):
        super(ClassificationLoss, self).__init__()
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, outputs, targets):
        return self.loss_fn(outputs, targets)


In [8]:
class FingerprintClassifier(nn.Module):
    def __init__(self, image_size=128, embedding_size=512, num_classes=3):
        super(FingerprintClassifier, self).__init__()
        self.image_size = image_size

        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),  # 1 input channel for grayscale
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv_output_size = self._get_conv_output_size()

        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.conv_output_size, embedding_size),
            nn.ReLU(),
            nn.Linear(embedding_size, num_classes)  # No sigmoid for multi-class logits
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

    def _get_conv_output_size(self):
        dummy_input = torch.ones(1, 1, self.image_size, self.image_size)  # 1 channel for grayscale
        x = self.conv_layers(dummy_input)
        return x.numel()


In [18]:
def classify_images_in_directory(test_dir, model_path=model_file):
    # Load model
    model = FingerprintClassifier().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    # Collect image paths
    image_files = [
        os.path.join(test_dir, f)
        for f in os.listdir(test_dir)
        if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif'))
    ]

    if not image_files:
        print("No images found in the directory.")
        return

    # Run classification
    for img_path in sorted(image_files):
        image = Image.open(img_path).convert('L')
        image = transform(image).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(image)
            predicted_class = output.argmax(dim=1).item()
            print(f"{os.path.basename(img_path)} => {label_map[predicted_class]}")

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Category labels (adjust if more/less classes)
label_map = {
    0: 'class1_Arc',
    1: 'class2_Whorl',
    2: 'class3_Loop'
}

# Image transformation (should match training)
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [20]:
classify_images_in_directory(datadir)

class1_Arc_0001.png => class1_Arc
class1_Arc_0010.png => class1_Arc
class1_Arc_0010_v1.png => class1_Arc
class1_Arc_0010_v2.png => class1_Arc
class1_Arc_0010_v3.png => class1_Arc
class1_Arc_0022.png => class1_Arc
class1_Arc_0030.png => class1_Arc
class1_Arc_0032.png => class1_Arc
